# MIDITok - Tokenizing, Training (+Generating!)

## 1. Install & Import necessary libraries

In [ ]:
!pip install datasets transformers
!pip install miditok
!pip install symusic

In [ ]:
from miditok import MMM, MuMIDI, TokenizerConfig
from miditok.pytorch_data import DatasetTok, DataCollator
from pathlib import Path
from symusic import Score

## 2. Tokenizing

MidiTok 에서 제공하는 tokenizer 클래스로 tokenizer 생성

In [ ]:
# Creating a multitrack tokenizer configuration, read the doc to explore other parameters

config = TokenizerConfig(
    num_velocities=16, 
    use_chords=True, 
    use_programs=True,
    use_pitch_intervals=True
    )

TOKENIZER_NAME = MMM # MMM 토크나이저 사용
# TOKENIZER_NAME = MuMIDI # MuMIDI 토크나이저 사용
tokenizer = TOKENIZER_NAME(config)

미디 파일 하나를 시험삼아 토크나이징 해봅니다

In [ ]:
# Loads a midi, converts to tokens, and back to a MIDI

midi = Score("./jazz-chunked/003_20thcenturystomp_cleaned/1.mid")
tokens = tokenizer(midi)  # calling the tokenizer will automatically detect MIDIs, paths and tokens
converted_back_midi = tokenizer(tokens)  # PyTorch / Tensorflow / Numpy tensors supported

토크나이징 결과 확인

In [ ]:
# len(tokens)
tokens

토큰에서 다시 생성된 미디 파일 정보 확인

In [ ]:
converted_back_midi

학습할 미디 파일들의 경로 지정

In [ ]:
midi_paths = list(Path("jazz-chunked").glob("**/*.mid"))
len(midi_paths)

(Optional) BPE Tokenizer 학습시키려면 아래 셀 주석 해제 (단, 생성 결과물 별로 좋지 못하고 생성시 컨트롤이 힘들 것으로 추정됨)

In [ ]:
# tokenizer.learn_bpe(vocab_size=10000, files_paths=midi_paths) # 원하는 vocab_size 자유롭게 지정
# tokenizer.save_params(Path("tokenizer/tokenizer.json")) # 학습한 BPE 토크나이저를 json 형태로 저장

# Load the BPE-trained tokenizer
# tokenizer = MMM(params=Path('tokenizer/tokenizer.json')) # 기 학습된 토크나이저를 경로에서 불러올 수 있음

## 3. Train / Validation Dataset 구축

train / valid 데이터셋 split 및 shuffle

In [ ]:
# Split MIDI paths in train/valid/test sets

from random import shuffle

total_num_files = len(midi_paths)
num_files_valid = round(total_num_files * 0.1) # Validation 비율 자유롭게 변경
shuffle(midi_paths)
midi_paths_valid = midi_paths[:num_files_valid]
midi_paths_train = midi_paths[num_files_valid:]

##### ⏰ 데이터를 MidiTok Dataset 형태로 처리 (V100 서버 기준 재즈 데이터에 약 1분 소요)

In [ ]:
# Creates a Dataset and a collator to be used with a PyTorch DataLoader to train a model
dataset_train = DatasetTok(
    files_paths=midi_paths_train,
    min_seq_len=50,
    max_seq_len=1022,
    tokenizer=tokenizer,
)
dataset_valid = DatasetTok(
    files_paths=midi_paths_valid,
    min_seq_len=50,
    max_seq_len=1022,
    tokenizer=tokenizer,
)
collator = DataCollator(
    tokenizer["PAD_None"], tokenizer["BOS_None"], tokenizer["EOS_None"], copy_inputs_as_labels=True
)

## 4. HuggingFace와 호환 가능한 형태로 데이터셋 변경
❓ HuggingFace Trainer로 train 시키기 위해 아래 과정을 수행합니다.<br>
❗️ 아래 셀들이 조금 필요없는 작업일 수 있는데, 일단 구현을 우선으로 작성 해놓았습니다. 혹시 불필요한 과정 발견하시면 제보 부탁드립니다!

In [ ]:
from torch.utils.data import DataLoader

data_loader_train = DataLoader(dataset=dataset_train, collate_fn=collator)
data_loader_valid = DataLoader(dataset=dataset_valid, collate_fn=collator)
train_tokenized_songs = []
valid_tokenized_songs = []
for batch in data_loader_train:
    train_tokenized_songs.append(batch)
for batch in data_loader_valid:
    valid_tokenized_songs.append(batch)

In [ ]:
# valid_tokenized_songs[0]

In [ ]:
# make custom dataset
import torch
from torch.utils.data import Dataset, DataLoader

class MidiDataset(Dataset):
    def __init__(self, tokenized_songs, max_length=510):  # max_length를 512로 하면 앞, 뒤에 BOS, EOS 토큰이 또 붙어서 길이 514 되고 에러가 나서 일단 510로 함. 디버깅 필요!!
        self.tokenized_songs = tokenized_songs
        self.max_length = max_length
    
    def __len__(self):
        return len(self.tokenized_songs)
    
    def __getitem__(self, idx):
        # item = {key: val.clone().detach() for key, val in self.tokenized_songs[idx].items()}
        item = {'input_ids': self.tokenized_songs[idx]['input_ids'][:, :self.max_length].clone().detach().squeeze(),}
        return item

In [ ]:
train_dataset = MidiDataset(train_tokenized_songs)
eval_dataset = MidiDataset(valid_tokenized_songs)

In [ ]:
train_dataset[0]['input_ids'].shape
# train_dataset[0]

In [ ]:
# Test our data_collator
out = collator([train_dataset[i] for i in range(5)])

for key in out:
    print(f"{key} shape: {out[key].shape}")

print(f"out {out}")

## 4. Training

Custom Trainer 정의

In [ ]:
from transformers import Trainer, TrainingArguments

# first create a custom trainer to log prediction distribution
class CustomTrainer(Trainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    def evaluation_loop(
        self,
        dataloader,
        description,
        prediction_loss_only=None,
        ignore_keys=None,
        metric_key_prefix="eval",
    ):
        # call super class method to get the eval outputs
        eval_output = super().evaluation_loop(
            dataloader,
            description,
            prediction_loss_only,
            ignore_keys,
            metric_key_prefix,
        )

        return eval_output

세부 training config 정하고, GPT-2 모델 불러오기

In [ ]:
from transformers import AutoConfig, GPT2LMHeadModel

context_length = 1024 # context length는 자유롭게 바꿔보며 실험해봐도 좋을 듯 합니다.

# Change this based on size of the data
n_layer=6
n_head=4
n_emb=1024

config = AutoConfig.from_pretrained(
    "gpt2",
    vocab_size=len(tokenizer),
    n_positions=context_length,
    n_layer=n_layer,
    n_head=n_head,
    pad_token_id=tokenizer["PAD_None"],
    bos_token_id=tokenizer["BOS_None"],
    eos_token_id=tokenizer["EOS_None"],
    n_embd=n_emb
)

model = GPT2LMHeadModel(config)
model

Training Argument 설정

In [ ]:
# Create the args for out trainer
from argparse import Namespace

# Get the output directory with timestamp.
output_path = "models"
steps = 100
# Commented parameters correspond to the small model
config = {"output_dir": output_path,
          "num_train_epochs": 30, # 학습 epoch 자유롭게 변경. 저는 30 epoch 걸어놓고 early stopping 했습니다.
          "per_device_train_batch_size": 32,
          "per_device_eval_batch_size": 16,
          "evaluation_strategy": "steps",
          "save_strategy": "steps",
          "eval_steps": steps,
          "logging_steps":steps,
          "logging_first_step": True,
          "save_total_limit": 5,
          "save_steps": steps,
          "lr_scheduler_type": "cosine",
          "learning_rate":5e-4,
          "warmup_ratio": 0.01,
          "weight_decay": 0.01,
          "seed": 1,
          "load_best_model_at_end": True,
          # "metric_for_best_model": "eval_loss" # best model 기준 바꾸고 싶을 경우 이 부분 변경 (default가 eval_loss임)
        #   "report_to": "wandb"
          }

args = Namespace(**config)

In [ ]:
from transformers import set_seed
set_seed(args.seed)

In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback

# mps device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

train_args = TrainingArguments(**config)

trainer = CustomTrainer(
    model=model,
    tokenizer=tokenizer,
    args=train_args,
    data_collator=collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=5)] # Early Stopping patience 자유롭게 변경
)

#### 드디어! 학습을 시작하세요! 
- V100 서버 기준 3 epoch 약 7분 소요
- 저는 30 epoch 정도 걸어놓고 Early Stopping 했습니다.
- 학습 완료된 후 모델 체크포인트는 models 폴더 안에 저장됩니다.

In [ ]:
# Train the model.
trainer.train()

## 5. GENERATE

처음 입력할 토큰을 지정해주고, tensor로 바꾸어 generated_ids 변수에 할당합니다

In [ ]:
initial_token = "BOS_None" # 시작 토큰을 BOS로 설정해줍니다.

In [ ]:
generated_ids = torch.tensor([[tokenizer[initial_token]]])
generated_ids

- iteration number와 현재 시간을 초기화합니다. 
- ts 변수에 저장된 시간 정보는 지금은 안 쓰이고 있는데, 파일명 등에 사용하시면 나중에 모니터링 하실 때 좋습니다.

In [ ]:
# Timecode 및 iteration number 초기화
import datetime

iteration_number = 0
ts = datetime.datetime.now().strftime("%y%m%d%H%M%S")

- 아래 셀을 실행하여 생성하세요.
- 여러 번 실행하면 실행 할 때마다 트랙이 추가됩니다.

In [ ]:
# Iteration 몇 번 돌았는지 기록
iteration_number += 1
print(f"current iteration : {iteration_number}")

# Encode the conditioning tokens.
input_ids = generated_ids.cuda() # 로컬에서 실행할 때는 cuda() 없애주기

# Generate more tokens.
eos_token_id = tokenizer["Track_End"] # "Track_End" 토큰이 나올 때까지 생성 => iteration당 악기 한 트랙씩 생성하는 원리
temperature = 0.8 # Temperature를 높이면 생성 결과가 더욱 randomize 되는 것 같습니다.
generated_ids = model.generate(
    input_ids,
    max_length=1024,
    do_sample=True,
    temperature=temperature,
    eos_token_id=eos_token_id,
).cpu()

print(generated_ids)

생성된 토큰 미디 데이터로 변환

In [ ]:
midi = tokenizer.tokens_to_midi(generated_ids[0])
midi

변환된 미디 데이터로 test_gen.mid 파일 생성

In [ ]:
midi.dump_midi(f'./test_gen_iter_{iteration_number}.mid')

### 🎉 축하합니다!

- 학습 및 생성을 완료하였습니다. 여러 세팅으로 학습해보시고, 생성된 미디 파일을 다운받아 https://bandlab.com/ 등에서 실행시켜보세요
- 서버에서 미디 재생이 안 되기 때문에, 생성된 모델과 miditok-gen.ipynb 파일을 다운받아 로컬에서 생성 실험을 계속하시는 것을 추천드립니다!